# ABS Inflation model

Aim: to forecasr the next trimmed mean (TM) print based on ...
- upstream inflation PPI
- prevaiing cost pressures WPI
- the Phillips curve (inflation and unemployment have an inverse relationship) UER)
- previous quarter TM, as the series is autocorrelated. 

## Python set-up

In [1]:
# system imports
from pathlib import Path
from typing import Sequence, cast

# analytic imports
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm 


In [2]:
# local imports
from abs_data_capture import (
    AbsLandingPage,
    AbsMultiSeries,
    AbsSelectInput,
    AbsSelectionDict,
    get_multi_series,
)

In [3]:
# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100


## Data capture

In [4]:
wanted: AbsSelectionDict = {
    "Qrtly CPI Trimmed Mean": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="consumer-price-index-australia",
        ),
        table="8",
        orig_sa="SA",
        search1="Percentage Change from Corresponding Quarter of Previous Year",
        search2="Trimmed Mean ;  Australia",
        abbr="CPI",
        calc_growth=False,
    ),
    "Producer Price Index": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="producer-price-indexes-australia",
        ),
        table="1",
        orig_sa="Orig",
        search1="Percentage change from corresponding quarter of previous year",
        search2="Final ;  Total ",
        abbr="PPI",
        calc_growth=False,
    ),
    "Wage Price Index": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="wage-price-index-australia",
        ),
        table="1",
        orig_sa="SA",
        search1="Percentage Change From Corresponding Quarter of Previous Year",
        search2="Australia ;  Total hourly rates of pay excluding bonuses ;  "
        + "Private and Public ;  All industries ;",
        abbr="WPI",
        calc_growth=False,
    ),
    "Unemployment Rate": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="labour",
            parent_topic="employment-and-unemployment",
            topic="labour-force-australia",
        ),
        table="1",
        orig_sa="SA",
        search1="Unemployment rate ;",
        search2="Persons",
        abbr="UER",
        calc_growth=False,
    ),
}

In [27]:
dataset = get_multi_series(wanted, verbose=False)

tm = dataset["Qrtly CPI Trimmed Mean"].series
tm1 = tm.copy()
tm1.index += 1  # shifted because TM is autocorrelated

uer = dataset["Unemployment Rate"].series
uer = uer[uer.index.month.isin([3, 6, 9, 12])].to_timestamp("M").to_period(freq="Q")
uer_delta = uer.diff(1)
uer_delta

wpi = dataset["Wage Price Index"].series
wpi.index += 1  # shifted because WPI is printed after the CPI
wpi_delta = wpi.diff(1)

ppi = dataset["Producer Price Index"].series
ppi.index += 1  # shifted because PPI is printed after the CPI
ppi_delta = ppi.diff(1)

y = tm[X_all.index]

data = pd.DataFrame([y, tm1, uer, uer_delta, wpi, wpi_delta, ppi, ppi_delta]).T
data.columns = ["y", "TM-1", "UER", "UER Delta", "WPI", "WPI Delta", "PPI", "PPI Delta"]
data_nona = data.dropna()
data_nona = sm.add_constant(data_nona)
assert data_nona.isna().sum().sum() == 0    

# display(data_nona)

## Build a simple multiple regression model

In [24]:

model_1 = sm.OLS(data_nona['y'], data_nona.loc[:, data_nona.columns != 'y'])
fit_1 = model_1.fit()
print(fit_1.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.927
Model:                            OLS   Adj. R-squared:                  0.921
Method:                 Least Squares   F-statistic:                     152.1
Date:                Fri, 03 May 2024   Prob (F-statistic):           6.26e-45
Time:                        12:32:24   Log-Likelihood:                -18.178
No. Observations:                  92   AIC:                             52.36
Df Residuals:                      84   BIC:                             72.53
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0875      0.280      3.885      0.0

In [29]:
# remodel based on significant items above, remove co-efficients near zero
model_2 = sm.OLS(data_nona['y'], data_nona[['TM-1', 'const', 'UER']])
fit_2 = model_2.fit()
print(fit_2.summary())

# note:simpled model with comparable R-squared, and iproved AIC, and BIC

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.921
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     513.7
Date:                Fri, 03 May 2024   Prob (F-statistic):           2.95e-49
Time:                        12:41:26   Log-Likelihood:                -21.882
No. Observations:                  91   AIC:                             49.76
Df Residuals:                      88   BIC:                             57.30
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TM-1           0.9651      0.033     29.437      0.0

## Finished

In [7]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Fri May 03 2024 12:04:46

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 8.22.2

statsmodels: 0.14.1
matplotlib : 3.8.4
pandas     : 2.2.2

Watermark: 2.4.3



In [8]:
print("Finished")

Finished
